In [81]:
import sys

sys.path.insert(1, "/Users/minhpham/Documents/Work/Python/Qiskit/New Project/FrozenYoghourt/")

from FrozenYoghourt import *
from FrozenYoghourt import *
from FrozenYoghourt.mode import *
from FrozenYoghourt.gates import *
from FrozenYoghourt.gates.single import *
from FrozenYoghourt.gates.double import *
from FrozenYoghourt.gates.multi import *
from FrozenYoghourt.maths import *
from FrozenYoghourt.quantum import *
from FrozenYoghourt.quantum.decomposition import *
from FrozenYoghourt.circuit import *
from FrozenYoghourt.visualization import *

import numpy as np
import sys

sys.path.insert(1, "/Users/minhpham/Documents/Research/laughing-umbrella/xx_synthesis/monodromy")

import monodromy

from monodromy.coordinates import unitary_to_monodromy_coordinate, monodromy_to_positive_canonical_coordinate

from matplotlib import pyplot as plt

from itertools import product
from functools import reduce

In [2]:
def generate_random_a1a2():
    # Generate a random value for y in the range [0, pi/4]
    a2 = np.random.uniform(0, np.pi / 4)
    
    # Generate a random value for x in the range [y, pi/2 - y]
    low, high = np.sort([a2, np.pi / 2 - a2])
    a1 = np.random.uniform(low, high)
    
    if a1>np.pi/4:
        a1 = np.pi/2-a1
        a2, a1 = np.sort([a1, a2]) # Since a3 = 0.
    
    assert a1+a2 <= np.pi/2, "a1 + a2 is greater than pi/2"
    assert a1>=a2, "a2 is greater than a1"
    assert a2 >=0, "a2 is greater than 0"
    assert a1 <= np.pi/4, "a1 is greater than pi/4"
    
    return a1, a2

def canonical_coordinate(mat):
    # Compute canonical coordinate
    return monodromy_to_positive_canonical_coordinate(*unitary_to_monodromy_coordinate(mat)[:3])

I'm gonna try to swap on Z gates for X gates to see if I get anything interesting

In [32]:
Mode.toggle("n")

a1, a2, d, e, beta = *generate_random_a1a2(), *np.random.uniform(0, np.pi, 2), np.random.uniform(0, np.pi/4)

f1, f2 = *np.random.uniform(0, np.pi, 1), 0
xy_circ = NumericalCircuit(2)
xy_circ.can(2*a1, 2*a2, 0)
xy_circ.ry(2*d, 0)
xy_circ.rz(2*e, 1)

# Add new term
xy_circ.rx([2*f1, 2*f2], [0, 1])

xy_circ.can(2*beta, 2*beta, 0)

xy_circ.unitary

NUMERICAL activated


array([[-0.21505452+0.08635128j,  0.72619165-0.51969731j,
         0.22011815+0.30757897j,  0.02837412+0.07066466j],
       [ 0.3282832 +0.11403868j, -0.04209379-0.09479627j,
        -0.16438448+0.14738758j, -0.57730358+0.69745417j],
       [ 0.57730358+0.69745417j,  0.16438448+0.14738758j,
        -0.04209379+0.09479627j,  0.3282832 -0.11403868j],
       [-0.02837412+0.07066466j, -0.22011815+0.30757897j,
         0.72619165+0.51969731j, -0.21505452-0.08635128j]])

In [33]:
canonical_coordinate(xy_circ.unitary)

(0.7657243963928414, 0.4029125178042192, 2.615901373506474e-16)

It seems like if we have

$$CAN(a_1, a_2, 0) \cdot \text{ three rotation gates } \cdot XY(\beta),$$

then we also get something that is locally equivalent to

$$CAN(b_1, b_2, 0)$$

Is this actually true?

For pauli matrices $A$ and $B$, we have $[A, B] = 2AB$

In [110]:
# Create the multiplication dictionary
pauli_dict = {
    'II': 'I',
    'IX': 'X',
    'IY': 'Y',
    'IZ': 'Z',
    'XI': 'X',
    'XX': 'I',
    'XY': 'iZ',
    'XZ': '-iY',
    'YI': 'Y',
    'YX': '-iZ',
    'YY': 'I',
    'YZ': 'iX',
    'ZI': 'Z',
    'ZX': 'iY',
    'ZY': '-iX',
    'ZZ': 'I'
}

phase_dict = {"": "", "i": "i", "-i": "-i", "ii" :"-", "-i-i" :"-", "i-i": "", "-ii": ""}

# Print the dictionary
print(pauli_dict)

{'II': 'I', 'IX': 'X', 'IY': 'Y', 'IZ': 'Z', 'XI': 'X', 'XX': 'I', 'XY': 'iZ', 'XZ': '-iY', 'YI': 'Y', 'YX': '-iZ', 'YY': 'I', 'YZ': 'iX', 'ZI': 'Z', 'ZX': 'iY', 'ZY': '-iX', 'ZZ': 'I'}


In [111]:
two_qubit_products = [reduce(lambda a, b: a+b, elem) for elem in \
                     list(product(["I", "X", "Y", "Z"], repeat = 4))]

In [137]:
pauli_multiplication_table = dict()

for pairs in two_qubit_products:
    A0, A1, B0, B1 = pairs
    C0L, C1L = pauli_dict[A0+B0], pauli_dict[A1+B1]
    phase0 = phase_dict[C0L[:-1]+C1L[:-1]]
    C0L, C1L = C0L[-1], C1L[-1]
    
    C0R, C1R = pauli_dict[B0+A0], pauli_dict[B1+A1]
    phase1 = phase_dict[C0R[:-1]+C1R[:-1]]
    C0R, C1R = C0R[-1], C1R[-1]
    
    if abs(len(phase0)-len(phase1))==1:
        pauli_multiplication_table[pairs] = C0L+C1L
    else:
        pauli_multiplication_table[pairs] = "00"

In [146]:
def generating_algebra(generators):
    iterating_algebra = set(generators)
    
    while True:
        # Python doesn't like its iterate changing
        fixed_algebra = iterating_algebra.copy()
        
        for A in fixed_algebra:
            for B in fixed_algebra:
                
                # Iterate through and compute
                C = pauli_multiplication_table[A+B]
                
                # Add to existing set
                if C != "00":
                    iterating_algebra.add(C)
        
        if len(fixed_algebra) == len(iterating_algebra):
            print("Generation completes")
            return iterating_algebra

In [147]:
generators = ["XX", "YY", "ZI", "IZ"]
generating_algebra(generators)

Generation completes


{'IZ', 'XX', 'XY', 'YX', 'YY', 'ZI'}

In [76]:
# Define the Pauli matrices as strings
I = 'I'
X = 'X'
Y = 'Y'
Z = 'Z'

# List all possible two-qubit Pauli matrices
two_qubit_paulis = [a+b for a in [I, X, Y, Z] for b in [I, X, Y, Z]]

# Initialize an empty dictionary to store the commutators
commutator_dict = {}

# Define a function to compute the commutator of two Pauli matrices
def commutator(A, B):
    if A == B:
        return '0'
    elif (A == 'XX' and B == 'YY') or (A == 'YY' and B == 'XX'):
        return 'ZZ'
    elif (A == 'XX' and B == 'ZZ') or (A == 'ZZ' and B == 'XX'):
        return 'YY'
    elif (A == 'YY' and B == 'ZZ') or (A == 'ZZ' and B == 'YY'):
        return 'XX'
    elif (A == 'XY' and B == 'YX') or (A == 'YX' and B == 'XY'):
        return 'ZZ'
    elif (A == 'XZ' and B == 'ZX') or (A == 'ZX' and B == 'XZ'):
        return 'YY'
    elif (A == 'YZ' and B == 'ZY') or (A == 'ZY' and B == 'YZ'):
        return 'XX'
    else:
        return '0'

# Compute the commutators for all combinations of two-qubit Pauli matrices
for pauli1 in two_qubit_paulis:
    for pauli2 in two_qubit_paulis:
        key = pauli1 + pauli2
        commutator_result = commutator(pauli1, pauli2)
        commutator_dict[key] = commutator_result

# Print the dictionary


{'IIII': '0', 'IIIX': '0', 'IIIY': '0', 'IIIZ': '0', 'IIXI': '0', 'IIXX': '0', 'IIXY': '0', 'IIXZ': '0', 'IIYI': '0', 'IIYX': '0', 'IIYY': '0', 'IIYZ': '0', 'IIZI': '0', 'IIZX': '0', 'IIZY': '0', 'IIZZ': '0', 'IXII': '0', 'IXIX': '0', 'IXIY': '0', 'IXIZ': '0', 'IXXI': '0', 'IXXX': '0', 'IXXY': '0', 'IXXZ': '0', 'IXYI': '0', 'IXYX': '0', 'IXYY': '0', 'IXYZ': '0', 'IXZI': '0', 'IXZX': '0', 'IXZY': '0', 'IXZZ': '0', 'IYII': '0', 'IYIX': '0', 'IYIY': '0', 'IYIZ': '0', 'IYXI': '0', 'IYXX': '0', 'IYXY': '0', 'IYXZ': '0', 'IYYI': '0', 'IYYX': '0', 'IYYY': '0', 'IYYZ': '0', 'IYZI': '0', 'IYZX': '0', 'IYZY': '0', 'IYZZ': '0', 'IZII': '0', 'IZIX': '0', 'IZIY': '0', 'IZIZ': '0', 'IZXI': '0', 'IZXX': '0', 'IZXY': '0', 'IZXZ': '0', 'IZYI': '0', 'IZYX': '0', 'IZYY': '0', 'IZYZ': '0', 'IZZI': '0', 'IZZX': '0', 'IZZY': '0', 'IZZZ': '0', 'XIII': '0', 'XIIX': '0', 'XIIY': '0', 'XIIZ': '0', 'XIXI': '0', 'XIXX': '0', 'XIXY': '0', 'XIXZ': '0', 'XIYI': '0', 'XIYX': '0', 'XIYY': '0', 'XIYZ': '0', 'XIZI': '0'